In [35]:
from math import *
import numpy as np

## Forward Kinematics

In [36]:
def transform(alpha, a, d, theta):
    TF = np.matrix([[cos(theta), -sin(theta), 0, a], 
                    [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d],
                   [sin(theta)*sin(alpha), cos(theta)*sin(alpha), cos(alpha), cos(alpha)*d],
                   [0, 0, 0, 1]])
    return TF

In [37]:
T1 = transform(0, 0, 0, pi/3)
T2 = transform(-pi/2, 0, 0, pi/2)
T3 = transform(0, 80, 40, pi/3)
T4 = transform(-pi/2, 30, 70, pi/2)
T5 = transform(pi/2, 0, 0, pi/2)
T6 = transform(-pi/2, 0, 0, pi/3)

In [38]:
T = T1*T2*T3*T4*T5*T6
T

matrix([[  0.25      ,   0.4330127 ,  -0.8660254 , -65.13139721],
        [  0.4330127 ,   0.75      ,   0.5       , -32.81088913],
        [  0.8660254 ,  -0.5       ,   0.        , -34.37822174],
        [  0.        ,   0.        ,   0.        ,   1.        ]])

In [39]:
phi = atan2(T[0,1],T[0,0])
(phi*180)/pi

60.00000000000001

In [40]:
T1_6 = np.linalg.inv(T1)*T
T1_6

matrix([[  0.5       ,   0.8660254 ,   0.        , -60.98076211],
        [  0.        ,   0.        ,   1.        ,  40.        ],
        [  0.8660254 ,  -0.5       ,   0.        , -34.37822174],
        [  0.        ,   0.        ,   0.        ,   1.        ]])

## Inverse Kinematics

In [41]:
from math import acos, asin, atan2, sqrt

In [42]:
def inverse_kinematics(T):
    a1 = 0
    a2 = 80
    a3 = 40
    d1 = 0
    d4 = 70
    d6 = 0
    # Extract rotation matrix
    R = T[:3, :3]

    # Extract position vector
    p = T[:3, 3]

    # Joint angles (theta) for the Puma 560 robot
    theta = np.zeros(6)

    # Calculate theta1
    theta[0] = atan2(p[1], p[0])

    # Calculate theta3
    D = (p[0]**2 + p[1]**2 + (p[2] - d1)**2 - a2**2 - a3**2) / (2 * a2 * a3)
    theta[2] = atan2(-sqrt(1 - D**2), D)

    # Calculate theta2
    K1 = a2 + a3 * cos(theta[2])
    K2 = a3 * sin(theta[2])
    theta[1] = atan2(p[2] - d1, sqrt(p[0]**2 + p[1]**2)) - atan2(K2, K1)

    # Calculate theta4, theta5, and theta6
    R03_inv = np.linalg.inv(R[:3, :3])
    R36 = np.dot(R03_inv, T3[:3, :3])
    theta[3] = atan2(R36[2, 1], R36[2, 0])
    theta[4] = acos(R36[2, 2])
    theta[5] = atan2(-R36[1, 2], R36[0, 2])

    return theta

In [45]:
theta_inverse = inverse_kinematics(T)
print(theta_inverse)

[-2.67493794  0.06278919 -1.80728909  1.57079633  1.57079633  0.52359878]


In [47]:
for i in range(len(theta_inverse)):
    print((theta_inverse[i]*180)/pi)

-153.26265443670087
3.5975557643557297
-103.55003706682172
90.00000000000001
90.0
29.999999999999993
